In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore') # Hides warning
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore",category=UserWarning)


In [2]:
csv = "/home/onkar/1429_1.csv"
df = pd.read_csv(csv)
df.head(2)

,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
0,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,This product so far has not disappointed. My c...,Kindle,NaN,NaN,Adapter
1,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,great for beginner or experienced person. Boug...,very fast,NaN,NaN,truman


In [3]:
def sentiments(rating):
    if (rating == 5) or (rating == 4):
        return '1'
    elif (rating == 3) or (rating == 2) or (rating == 1):
        return '0'

In [9]:
def helpful(rating):
    if (rating > 1):
        return '1'
    return '0'

In [10]:
df["Sentiment"] = df["reviews.rating"].apply(sentiments)

df['reviews.text']=df['reviews.text'].astype(str)

In [11]:
df["helpful"] = df["reviews.numHelpful"].apply(helpful)

In [12]:
len(df[df["helpful"] == '0'])

33398

In [13]:
len(df[df["helpful"] == '1'])

1262

In [14]:
df.to_csv("ProductReviews.csv", header=True,index= False)

In [17]:
def getStemmedReview(reviews_text):
    reviews_text=reviews_text.lower()
    reviews_text=reviews_text.replace("<br /><br />"," ")
    #Tokenize
    tokens=tokenizer.tokenize(reviews_text)
    new_tokens=[token for token in tokens if token not in  en_stopwords]
    stemmed_tokens=[ps.stem(token) for token in new_tokens]
    clean_review=' '.join(stemmed_tokens)
    return clean_review

In [20]:
from sklearn.model_selection import StratifiedShuffleSplit
strat_train = df.dropna(subset=["reviews.rating"]) # removes all NAN in reviews.rating
strat_train = df.dropna(subset=["reviews.text"]) # removes all NAN in reviews.text

In [21]:
df['reviews.text'].apply(getStemmedReview)
X_train = strat_train["reviews.text"]
Y_train = strat_train["reviews.rating"]

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(sublinear_tf=True, encoding='utf-8',decode_error='ignore')
vectorizer.fit(X_train)
X_train=vectorizer.transform(X_train)

In [24]:
df.fillna(' ')
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train, Y_train.astype('U'))

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [25]:
print("Score on training data is: "+str(model.score(X_train,Y_train.astype('U'))))

Score on training data is: 0.703664166185805


In [26]:
import joblib
joblib.dump(model,'model.pkl')
joblib.dump(vectorizer,'vectorizer.pkl')

['vectorizer.pkl']